In [1]:
import tensorflow as tf

# 使用 CPU

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

my_devices = tf.config.experimental.list_physical_devices()
my_devices

No GPU found


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]

In [ ]:
tf.gather

In [ ]:
tf.cast

In [ ]:
tf.reshape

In [ ]:
tf.sequence_mask

# tf.slice(input_, begin, size, name=None)
beigin 为各个维度的起始索引点，size 为各个维度切片元素的 数量

In [2]:
a = tf.constant(np.random.randint(0, 5, size=(4, 4)))
print(a)
b = tf.slice(a, begin=[0, 1], size=[-1, 3])
print(b)

tf.Tensor(
[[0 0 4 3]
 [0 0 1 3]
 [2 4 4 4]
 [2 0 3 3]], shape=(4, 4), dtype=int64)
tf.Tensor(
[[0 4 3]
 [0 1 3]
 [4 4 4]
 [0 3 3]], shape=(4, 3), dtype=int64)


# tf.cond(pred, true_fn=None, false_fn=None, name=None)
pred 为真时，执行 true_fn() 函数，否则执行 false_fn() 函数

In [11]:
x = tf.constant([[1, 2, 3], [4, 1, 0]])


def f1():
    return x**2


def f2():
    return tf.reduce_sum(x, axis=-1)


tf.cond(tf.shape(x)[-1] != 3, f1, f2)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([6, 5], dtype=int32)>

# tf.scan(fn,elems,**keargs)

根据 `elems` 的第 0 维用 `fn`进行扫描；fn 接受两个参数，第一个参数为之前激活 fn 的累计值，第二个参数为 `elems` 的当前元素
> 特定情况下，可以方便实现动态规划

In [14]:
elems = np.array([1, 2, 3, 4, 5, 6])
sum = tf.scan(lambda a, x: a + x, elems)
sum

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([ 1,  3,  6, 10, 15, 21])>

# tf.tile((input, multiples, name=None))
将 input 的各个维度重复 multiples 上对应维度上数值

In [4]:
x = tf.constant([[1, 2, 3], [4, 1, 0]])
tf.tile(x, [2, 2])

<tf.Tensor: shape=(4, 6), dtype=int32, numpy=
array([[1, 2, 3, 1, 2, 3],
       [4, 1, 0, 4, 1, 0],
       [1, 2, 3, 1, 2, 3],
       [4, 1, 0, 4, 1, 0]], dtype=int32)>

# tf.einsum

转置:$B_{ji}=A_{ij}$

In [6]:
a = tf.random.uniform((2, 3))
a

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.8758718 , 0.4365437 , 0.38201618],
       [0.50707936, 0.5221009 , 0.5417745 ]], dtype=float32)>

In [7]:
b = tf.einsum("ij->ji", a)
b

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.8758718 , 0.50707936],
       [0.4365437 , 0.5221009 ],
       [0.38201618, 0.5417745 ]], dtype=float32)>

求和：$b=\sum\limits_{i}\sum\limits_{j}A_{ij}$

In [8]:
b = tf.einsum("ij->", a)
b

<tf.Tensor: shape=(), dtype=float32, numpy=3.2653863>

列求和：$b=\sum\limits_{i}A_{ij}$

In [10]:
b = tf.einsum("ij->j", a)
b

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.3829511, 0.9586446, 0.9237907], dtype=float32)>

矩阵乘法：$C_{ij} = \sum\limits_{k}A_{ik}B_{kj}$

In [16]:
a = tf.random.normal((3, 5))
b = tf.random.uniform((5, 4))
c = tf.einsum("ij,jk->ik", a, b)
c

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[-0.99365306,  0.24569835, -0.4555341 , -0.18225688],
       [-1.1832956 , -1.9588448 , -2.2628129 , -0.65367246],
       [-0.42967057,  0.86160624,  0.5546088 ,  0.45343527]],
      dtype=float32)>

In [18]:
# Dot product
u = tf.constant([1, 2, 3])
v = tf.constant([2, 3, 5])
tf.einsum('i,i->', u, v)  # output = sum_i u[i]*v[i]

<tf.Tensor: shape=(), dtype=int32, numpy=23>

In [19]:
# Outer product
tf.einsum('i,j->ij', u, v)  # output[i,j] = u[i]*v[j]

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 2,  3,  5],
       [ 4,  6, 10],
       [ 6,  9, 15]], dtype=int32)>

In [21]:
# Trace
m = tf.constant([[1, 2], [3, 4]])
tf.einsum('ii', m)  # output[j,i] = trace(m) = sum_i m[i, i]

<tf.Tensor: shape=(), dtype=int32, numpy=5>

In [23]:
# Batch matrix multiplication
s = tf.random.uniform((2, 3, 5))
t = tf.random.normal((2, 5, 7))
tf.einsum('aij,ajk->aik', s, t)  # out[a,i,k] = sum_j s[a,i,j] * t[a, j, k]

<tf.Tensor: shape=(2, 3, 7), dtype=float32, numpy=
array([[[-0.04714242, -0.58901787,  1.827867  ,  1.7002294 ,
          1.0728605 , -0.44502312, -0.7983513 ],
        [ 1.7442985 ,  0.05938584,  1.2370716 , -0.5335729 ,
          2.0766802 , -0.1455391 , -1.0797524 ],
        [ 0.7962718 , -0.8501762 ,  1.8877522 ,  0.7560884 ,
          1.1441047 ,  0.03892493, -0.98979926]],

       [[ 0.7783072 , -1.4198673 , -0.76826286, -1.3987927 ,
         -0.5849587 , -0.9192537 ,  1.018877  ],
        [ 1.4122456 , -2.0655475 , -1.4665165 , -0.4460135 ,
         -1.8070424 , -0.8549925 ,  0.23259544],
        [ 1.4935722 , -3.0562093 , -0.5718592 ,  0.70153415,
         -1.3151858 ,  0.08705962, -0.07307735]]], dtype=float32)>